# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [36]:
import json
import nltk
import os
import random
import re
import torch
import numpy as np


from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [37]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)
# print(len(twits))
twits['sentiment'] = np.random.randint(0,4, size=len(twits['data']))
print(twits['data'][:10])

[{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN', 'timestamp': '2018-11-01T00:00:05Z'}, {'message_body': '$MCD bt puts when Cramer touted going to $200 so for a profit and bt back yesterday down on up day is good $PEP looks lower too', 'timestamp': '2018-11-01T00:00:05Z'}, {'message_body': '$AAPL 190 after ER', 'timestamp': '2018-11-01T00:00:18Z'}, {'message_body': '$PYPL Insider &quot;Pentland Adele Louise&quot; just cashed in 8,149 Options (Tran Code - M, S)  --- Insider Forms, iOS app', 'timestamp': '2018-11-01T00:00:25Z'}, {'message_body': '$AMZN A lot of technical signs point to a short-term bottom in place for stocks (John Murphy)', 'timestamp': '2018-11-01T00:00:42Z'}, {'message_body': '$AAPL https://stockcharts.com/freecharts/pnf.php?c=aapl,PWTAWANRBO[PA][D][F1!3!!!2!20]', 'timestamp': '2018-11-01T00:00:55Z'}, {'message_body': '$NFLX Option Order Flow Sentiment is 60.9% Bullish. http://tinyurl.com/y6v6lnmz', 

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [38]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


87343


### Split Message Body and Sentiment Score

In [39]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
# sentiments = [twit['sentiment'] + 2 for twit in twits['data']]
sentiments = twits['sentiment']

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [40]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hxi00\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
#     text = re.sub(r'http\:\/\/\S+', " ", text, flags=re.MULTILINE)
    text = re.sub(r'http://\S+', " ", text, flags=re.MULTILINE)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\w*', " ", text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@\w*', " ", text)

    # Replace everything not a letter with a space
    text = re.sub(r"[^a-zA-Z0-9]+", " ", text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')  
#     words = word_tokenize(text)
#     print(tokens)
    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()   # filter away empty tokens
    tokens = [wnl.lemmatize(w) for w in tokens if len(w) > 1]
#     print(tokens)
    return tokens

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [50]:
# TODO Implement

new_messages = []
for index in range(len(messages)):
    new_message = preprocess(messages[index])
    if new_message:
        new_messages.append(new_message)

tokenized = new_messages
new_messages[:5]


# tokenized = [preprocess(message) for message in messages]

['ha', 'moved', '69', 'on', '10', '31', 'check', 'out', 'the', 'movement', 'and', 'peer', 'at', 'http', 'dividendbot', 'com', 'jwn']
['bt', 'put', 'when', 'cramer', 'touted', 'going', 'to', 'so', 'for', 'profit', 'and', 'bt', 'back', 'yesterday', 'down', 'on', 'up', 'day', 'is', 'good', 'look', 'lower', 'too']
['190', 'after', 'er']
['insider', 'quot', 'pentland', 'adele', 'louise', 'quot', 'just', 'cashed', 'in', '149', 'option', 'tran', 'code', 'insider', 'form', 'io', 'app']
['lot', 'of', 'technical', 'sign', 'point', 'to', 'short', 'term', 'bottom', 'in', 'place', 'for', 'stock', 'john', 'murphy']
['http', 'stockcharts', 'com', 'freecharts', 'pnf', 'php', 'aapl', 'pwtawanrbo', 'pa', 'f1', '20']
['option', 'order', 'flow', 'sentiment', 'is', '60', 'bullish']
['long', 'only', 'and', 'long', 'amp', 'put', 'short', 'and']
['held', 'while', 'dove', 'at', 'close', 'solid', 'lunch', 'bid', 'and', 'hold', 'on', 'prior', 'low', 'will', 'look', 'to', 'unload', 'my', '10', 'call', 'either', '

['max', 'pain', 'is', '135', 'for', 'expiry', '2018', '11', '02', 'source']
['bought', '105', 'call', 'exp', 'in', 'january', 'earlier', 'in', 'the', 'day', 'think', 'made', 'big', 'mistake']
['swing', 'blocking']
['once', 'rockstar', 'ha', 'been', 'littered', 'with', 'put', 'sweeper', 'for', 'month', 'put', 'action', 'again', 'this', 'morning', 'doe', 'have', 'earnings', 'coming']
['really', 'bug', 'me', 'we', 'went', 'down', 'price', 'and', 'our', 'volume', 'wa', 'so', 'high', 'at', 'end', 'of', 'day']
['unusual', 'put']
['max', 'pain', 'is', '135', 'for', 'expiry', '2018', '11', '16', 'source']
['amp', '500', 'future', 'dec', '18', 'short', 'future', 'green', 'in', 'thur', 'asia']
['unusual', 'call']
['hopefully', 'drop', 'to', '45']
['why', 'would', 'apple', 'all', 'of', 'sudden', 'have', 'bad', 'quarter']
['staanalystalert', 'for', 'citigroup', 'maintains', 'with', 'rating', 'of', 'buy', 'setting', 'target', 'price', 'at', 'usd', '45', '00', 'our', 'own', 'verdict', 'is', 'buy']
[

['imagine', 'buyout', 'their', 'stock', 'price', 'is', 'will', 'shareholder', 'be', 'worth', 'after', 'buyout']
['po']
['the', 'eps', 'growth', 'is', 'accelerating', 'in', 'the', 'next', 'year', 'the', 'growth', 'will', 'be', 'better', 'than', 'in', 'the', 'last', 'year', 'http', 'www', 'chartmill', 'com', 'analyze', 'php', 'utm', 'source', 'stocktwits', 'amp', 'utm', 'medium', 'fa', 'amp', 'utm', 'content', 'growth', 'amp', 'utm', 'campaign', 'social', 'tracking', 'rtn', 'fa', 'amp', 'key', '316f7c65', '1c7b', '4ea8', 'b174', '149b2613a986']
['even', 'with', 'oil', 'the', 'oil', 'stock', 'look', 'like', 'death']
['how', 'high', 'can', 'we', 'expect', 'this', 'to', 'go', 'from', 'it', 'beat']
['double', 'bottom', 'identified', 'more', 'insight', 'http', 'stockinvest', 'u', 'technical', 'analysis', 'm', 'utm', 'source', 'stocktwits', 'amp', 'utm', 'medium', 'post']
['39', 'trying', 'to', 'buy', 'but', 'it', 'won', '39', 'let', 'me']
['break', '280', 'then', 'free', 'fall', 'from', 'ther

['swinging', 'for', 'the', 'fence', 'tomorrow']
['staanalystalert', 'for', 'credit', 'suisse', 'upgrade', 'with', 'rating', 'of', 'neutral', 'our', 'own', 'verdict', 'is', 'under', 'perform']
['holy', 'grail', 'pump', 'into', 'friday', 'and', 'this', 'bad', 'baby', 'is', 'sitting', 'on', 'split', 'open', 'gap', 'ready', 'to', 'explode', 'to', '83', '77', '08', 'cash', 'out']
['based', 'on', 'stats', '74', 'chance', 'will', 'close', 'above', '82', 'support', 'by', 'end', 'of', 'session', 'bullish']
['max', 'pain', 'is', '73', 'for', 'expiry', '2018', '11', '02', 'source']
['just', 'noticed', 'that', 'analyst', 'estimate', 'for', 'aapl', 'for', 'next', 'qtr', 'not', 'tomorrow', 'are', 'for', 'almost', '00', 'share', 'terrific', 'news', 'for', 'mu', 'imo']
['core', 'of', 'amzn', 'sale', 'come', 'from', '3rd', 'party', 'seller', 'amzn', 'hold', 'fund', 'for', 'week', 'billion', 'in', 'fund', 'amazon', 'wks', 'away', 'from', 'br', 'everyday']
['max', 'pain', 'is', '116', 'for', 'expiry', '2

['the', 'current', 'ratio', 'of', 'is', 'much', 'better', 'than', 'the', 'industry', 'average', 'of', '15', 'http', 'www', 'chartmill', 'com', 'analyze', 'php', 'utm', 'source', 'stocktwits', 'amp', 'utm', 'medium', 'fa', 'amp', 'utm', 'content', 'health', 'amp', 'utm', 'campaign', 'social', 'tracking', 'fa', 'amp', 'key', '94343187', 'c937', '44e5', 'bcc2', 'ab18a48e7785']
['573', 'is', 'price', 'target', 'for', 'friday', '551', 'for', 'tomorrow']
['don', '39', 'have', 'good', 'feeling', 'about', 'er', 'guess', 'what', 'it', 'doesn', '39', 'matter', 'what', 'feel', 'price', 'action', 'amp', 'demand', 'is', 'all', 'that', 'matter', 'tariff', 'too']
['super', 'sniper', 'trade', 'by', 'jared', 'on', 'today', 'minute', 'for', 'over', 'http', 'www', 'youtube', 'com', 'watch', '82az32ko8pg', 'amp', 'feature', 'youtu', 'be']
['earnings', 'report', 'is', 'in', 'the', 'morning', 'tomorrow']
['love', 'this', 'one']
['max', 'pain', 'is', '111', 'for', 'expiry', '2018', '11', '02', 'source']
['a'

...

['million', 'traded', 'at', '93', '85']
['in', 'world', 'of', 'instagram', 'snapchat', 'fb', 'and', 'tiniest', 'bikini', 'there', 'ha', 'never', 'been', 'better', 'time', 'to', 'have', 'cellulite', 'treatment', '20', 'b4', '25']
['staanalystalert', 'for', 'argus', 'maintains', 'with', 'rating', 'of', 'buy', 'setting', 'target', 'price', 'at', 'usd', '155', '00', 'our', 'own', 'verdict', 'is', 'buy']
['short', 'watch', 'intensified', 'etc', 'them', 'wynn', 'put', 'though', 'amp', 'sq', 'naked', 'short', 'humbled']
['here', 'is', 'why', 'never', 'buy', 'anything', 'when', 'it', 'wa', '198', 'they', 'say', '190', '184', 'now', 'it', 'may', 'revisit', 'the', 'low', 'they', 'will', 'say', '180', 'amp', 'never', 'get', 'it']
['since', 'my', 'post', 'over', '128', 'profit', 'per', 'share', 'thanks', 'god']
['max', 'pain', 'is', '26', 'for', 'expiry', '2018', '11', '16', 'source']
['reported', 'yoy', 'growth', 'in', 'revpar', 'for', 'q3']
['file', 'form', '10', 'http', 'fintel', 'io', 'filing'

['dude', 'most', 'of', 'u', 'on', 'this', 'board', 'didn', 'gave', 'up', 'our', 'position', 'when', 'sp', 'at', '52', 'week', 'low', 'do', 'you', 'think', 'we', 'are', 'scared']
['short', 'sale', 'volume', 'not', 'short', 'interest', 'for', 'at', '2018', '11', '06', 'is', '50', 'via']
['coke', 'owning', '20', 'of', 'and', 'still', 'wanting', 'to', 'roll', 'out', 'their', 'own', 'energy', 'drink', 'doesn', '39', 'indicate', 'that', 'they', 'want', 'to', 'buy', 'more']
['doe', 'deserve', 'to', 'be', 'at', 'ath', 'with', 'this', 'reality', 'the', 'selloff', 'will', 'be', 'fast', 'furious', 'mcdonald', '39', 'overseas', 'strength', 'counter', 'sale', 'miss', 'http', 'www', 'reuters', 'com', 'article', 'u', 'mcdonalds', 'result', 'mcdonalds', 'overseas', 'strength', 'counter', 'sale', 'miss', 'iduskcn1mx1pj', 'il']
['mark', 'it', 'take', 'meesopho', 'too']
['stock', 'rally', 'build', 'in', 'asia', 'after', 'election', 'hurdle', 'market', 'wrap', 'via']
['max', 'pain', 'is', '83', 'for', 'ex

['retail', 'never', 'learn', 'this', 'is', 'total', 'loser', 'stock', 'if', 'want', 'good', 'idea', 'what', 'will', 'happen', 'just', 'look', 'at', 'the', 'history', 'called', 'it', 'short', 'b4', 'last', 'r']
['back', 'to']
['compared', 'to', 'an', 'average', 'industry', 'pe', 'ratio', 'of', '16', '13', 'is', 'valued', 'rather', 'cheaply', 'http', 'www', 'chartmill', 'com', 'analyze', 'php', 'utm', 'source', 'stocktwits', 'amp', 'utm', 'medium', 'fa', 'amp', 'utm', 'content', 'valuation', 'amp', 'utm', 'campaign', 'social', 'tracking', 'kor', 'fa', 'amp', 'key', 'b22a4e6b', '22cd', '484c', '8de9', '058bac34aa65']
['beat', 'the', 'estimize', 'eps', 'consensus', 'by', '18c', 'and', 'the', 'estimize', 'revenue', 'consensus', 'by', '78m', 'report', 'fq3', 'earnings', 'of', '31', 'eps', 'and']
['just', 'bought', '100']
['if', 'is', 'buy', 'up', 'here', 'at', 'bubble', 'ath', 'why', 'are', 'insider', 'exercising', 'their', 'option', 'and', 'selling', 'just', 'matter', 'of', 'time', 'before'

['percent', 'short', 'volume', 'for', 'wa', '46', 'on', '11', '07', '18', 'http', 'volumebot', 'com', 'rig']
['core', 'exploration', 'ltd', 'asx', 'cxo', '2018', 'agm', 'chairman', '39', 'address', 'abnnewswire']
['are', 'you', 'sticking', 'around', 'this', 'time', 'last', 'time', 'listen', 'to', 'everything', 'you', 'said', 'then', 'just', 'disappeared']
['flagn', '200', 'dma', 'target']
['large', 'cap', '55', 'technical', 'signal', 'bullish', 'bearish', 'more', 'signal', 'http', 'arizet', 'com', 'id', 'z20', 'tk', 'mlm']
['tomorrow', 'all', 'casino', 'stock', 'may', 'get', 'roughed', 'up', 'at', 'first', 'sector', 'is', 'down', 'and', 'the', 'sell', 'off', 'is', 'nasty', 'ah']
['max', 'pain', 'is', '20', 'for', 'expiry', '2018', '11', '09', 'source']
['where', 'are', 'those', 'moron', 'that', 'said', 'amzn', 'wa', 'going', 'under', '000']
['she', 'never', 'fails', 'the', 'one', 'that', 'always', 'come', 'through']
['so', 'much', 'for', 'comcast', 'buying', 'roku']
['ah', 'volume', 'o

[['ha',
  'moved',
  '69',
  'on',
  '10',
  '31',
  'check',
  'out',
  'the',
  'movement',
  'and',
  'peer',
  'at',
  'http',
  'dividendbot',
  'com',
  'jwn'],
 ['bt',
  'put',
  'when',
  'cramer',
  'touted',
  'going',
  'to',
  'so',
  'for',
  'profit',
  'and',
  'bt',
  'back',
  'yesterday',
  'down',
  'on',
  'up',
  'day',
  'is',
  'good',
  'look',
  'lower',
  'too'],
 ['190', 'after', 'er'],
 ['insider',
  'quot',
  'pentland',
  'adele',
  'louise',
  'quot',
  'just',
  'cashed',
  'in',
  '149',
  'option',
  'tran',
  'code',
  'insider',
  'form',
  'io',
  'app'],
 ['lot',
  'of',
  'technical',
  'sign',
  'point',
  'to',
  'short',
  'term',
  'bottom',
  'in',
  'place',
  'for',
  'stock',
  'john',
  'murphy']]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [57]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
all_twits = lambda t: [twt for tokenized in t for twt in tokenized]
all_twits = all_twits(tokenized)
print(all_twits[:10])
bow = Counter(all_twits)
# print(bow)

['ha', 'moved', '69', 'on', '10', '31', 'check', 'out', 'the', 'movement']


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [60]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {key: value/len(bow) for key, value in bow.items()}
# print(freqs)

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5E-5

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = bow.most_common(n=high_cutoff)   # http://www.pythoner.com/205.html


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

[('the', 23528), ('to', 20609), ('is', 17326), ('for', 17197), ('http', 13452), ('and', 12892), ('in', 12277), ('on', 11962), ('com', 11715), ('of', 11700), ('it', 10868), ('this', 10260), ('39', 10048), ('amp', 8425), ('11', 7648), ('will', 7506), ('www', 7255), ('at', 6839), ('utm', 6802), ('up', 6370)]


16907

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [61]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:ii for ii, word in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {word: ii for word,ii in enumerate(filtered_words,1)}
# tokenized with the words not in `filtered_words` removed.
from tqdm import tqdm
filtered = [[word for word in message if word in vocab] for message in tqdm(tokenized)]

100%|████████████████████████████████████████████████████████████████████████| 85472/85472 [00:00<00:00, 249847.76it/s]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [62]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    try:
        message = filtered[idx]
    except:
        continue

    if len(message) == 0:
        continue   # skip this message because it has length zero
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [63]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.201213801371146

Finally let's convert our tokens into integer ids which we can pass to the network.

In [66]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']
print(token_ids[:10])
print(sentiments[:10])

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23, 24, 25, 26, 11, 18, 27, 28, 29, 4, 30, 31, 32, 33, 34, 35, 36], [37, 38, 39], [40, 41, 42, 43, 44, 41, 45, 46, 47, 48, 49, 50, 51, 40, 52, 53, 54], [55, 56, 57, 58, 59, 23, 60, 61, 62, 47, 63, 25, 64, 65, 66], [14, 67, 16, 68, 69, 70, 71, 72, 73, 74, 75], [49, 76, 77, 78, 32, 79, 80], [81, 82, 11, 81, 83, 19, 60, 11], [84, 85, 86, 13, 87, 88, 89, 90, 11, 91, 4, 92, 93, 94, 34, 23, 95, 96, 5, 97, 98, 13, 99, 100, 101, 30], [102, 87]]
[3, 0, 3, 3, 0, 1, 3, 1, 3, 0]


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [67]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Setup additional layers

        self.lstm = nn.LSTM(embed_size,lstm_size,lstm_layers,dropout=dropout,batch_first=False) # LSTM layer

        self.dropout = nn.Dropout(dropout) # Dropout layer

        self.fc = nn.Linear(lstm_size, output_size) # Liner
        self.logsoftmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                        weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_()) 

        return hidden_state


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        nn_input = nn_input.long()

        # embeded words
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds,hidden_state)

        lstm_out = lstm_out[-1,:,:]
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # softmax function
        logps = self.logsoftmax(out)

        return logps, hidden_state

### View Model

In [73]:
# __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.6425, -1.7583, -1.4522, -1.5196, -1.7075],
        [-1.5938, -1.8206, -1.4370, -1.5272, -1.7140],
        [-1.6343, -1.7620, -1.4565, -1.5185, -1.7085],
        [-1.6279, -1.7692, -1.4587, -1.5159, -1.7089]],
       grad_fn=<LogSoftmaxBackward>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [74]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [75]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

split = int(len(token_ids) * .8 )
#print(split[:5])

train_features,remaining_x = token_ids[:split],token_ids[split:]

val  = int(len(token_ids[split:]) * .5)
valid_features = remaining_x[:val]

train_labels,remaining_y = sentiments[:split],sentiments[split:]
valid_labels = remaining_y[:val]
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

In [70]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(16908, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (logsoftmax): LogSoftmax(dim=1)
)

In [82]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
from time import time
import numpy as np

epochs = 1
batch_size = 1024
learning_rate = 0.001
clip = 5 # gradient clipping
print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    tic = time()
    print('Starting epoch {}'.format(epoch + 1))

    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        
        # break if the iteration takes too many steps
        if steps == 500:
            break
        
        hidden = model.init_hidden(batch_size)

        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        labels = labels.type(torch.LongTensor)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        hidden = tuple([each.data for each in hidden])

        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        logps, hidden = model(text_batch, hidden) 
        loss = criterion(logps, labels) # calculate the loss
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip) 
        optimizer.step()

        if steps % print_every == 0:
            start = time()
            model.eval()

            # TODO Implement: Print metrics
            # valid_hidden = model.init_hidden(batch_size)
            valid_hidden = model.init_hidden(batch_size)
            valid_losses = []
            accuracy = []
            for text_batch, labels in dataloader(
                    valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                if text_batch.size(1) != batch_size:
                    break 
                for each in hidden:
                    each.to(device)                    
                    valid_hidden = tuple([each.data for each in valid_hidden]) # backprop
                    text_batch, labels = text_batch.to(device), labels.to(device)
                    valid_logps, valid_hidden = model(text_batch, valid_hidden)
                    valid_loss = criterion(valid_logps.squeeze(), labels.long())
                    valid_losses.append(valid_loss.item())
                    # accuracy
                    ps = torch.exp(valid_logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())
            tac = time()
            model.train()
            print('Epoch: {}/{}...'.format(epoch+1, epochs)
                    ,' Step: {}...'.format(steps)
                    ,' Train Loss: {:.6f}...'.format(loss.item())
                    ,' Valid Loss: {:.6f}'.format(np.mean(valid_losses))
                    ,' Accuracy: {:.6f}'.format(np.mean(accuracy))
                    ,' Time: {}'.format((tac-start)))

    print('Epoch: {}'.format((time()-tic)))

Starting epoch 1


RuntimeError: Expected hidden[0] size (2, 575, 512), got [2, 1024, 512]

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [29]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement

    tokens = preprocess(text)

    # Filter non-vocab words
    tokens = [val for val in tokens if val in vocab]
    # Convert words to ids
    tokens = [vocab[x] for x in tokens] 

    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1, 1))) 
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)

    return pred

In [30]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[0.2081, 0.1879, 0.1975, 0.1984, 0.2081]], grad_fn=<ExpBackward>)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [32]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [33]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [34]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [35]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[0.2026, 0.1904, 0.1997, 0.1992, 0.2081]], grad_fn=<ExpBackward>),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.